In [32]:
import networkx as nx
from bs4 import BeautifulSoup
import re
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import requests
import os
import numpy as np
from operator import itemgetter
import dill as pickle
import plotly.graph_objects as go 
from collections import Counter

In [33]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [34]:
dataYears = ["18", "17", "16", "15", "14", "13"]
dataMonths = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
dataMonths2Dig = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
dataPath = Path("data")
dummyOver400Label = -1
dummyNotGMLabel = -2
ratingsCounterList = []
ratingsCounterListAll = []
gamesPlayedDictList = []

monthlyGraphsHashMap = {}
idList = []
nameRatingMap = {}

In [35]:
def monthDescriptiveLabel(cnt):
    yearIndex = cnt // 12
    monthIndex = cnt % 12
    descriptor = dataMonths[monthIndex]+"20"+dataYears[::-1][yearIndex]
    return descriptor
    
#print(monthDescriptiveLabel(33))

In [42]:
#from txt to csv, eliminating unnecessary data, after this piece of code is executed, the data for months is situated in aptly named folders dec13, jan14 etc.

def dataPreprocessing():
    dataMonthsRev = list(reversed(dataMonths)) # reversing the list because pre august 2016 data had a different organisation within the file
    changeInDataset = True
    ratingLimit = 2000
    ratingsCounterList.clear()
    ratingsCounterListAll.clear()
    
    for year in dataYears[::-1]:
        for month in dataMonths:
            
            ratingsCounter = Counter()
            ratingsCounterAll = Counter()
            
            folderName = month + year
            if folderName == "sep16":
                changeInDataset = False

            fileName = "standard_" + folderName + "frl.txt"
            with open(dataPath/folderName/fileName, 'r') as myfile:
                with open(dataPath/folderName/(folderName + ".csv"), 'w') as writeFile:
                    people = []
                    lines = myfile.readlines()
                    for line in tqdm(lines):
                        id = line[:15].strip()
                        
                        if id == 'ID Number': # eliminate file headers
                            continue
                        
                        name = line[15:76].strip()
                        name = name.replace(',', ';', 10) # replace all commas in name with semicolons, because comma is a delimiter in csv file
                        
                        nat = line[76:80].strip()
                        gender = line[80]
                        
                        if changeInDataset:
                            rating_str = line[109:115].strip()
                        else:
                            rating_str = line[113:118].strip()
                        
                        if len(rating_str) > 0:
                            rating = int(rating_str)
                            
                        if changeInDataset:
                            games_played = line[115:118].strip()
                        else:
                            games_played = line[119:122].strip()
            
#                         if len(games_played_str) > 0:
#                             games_played = int(games_played_str)
                        
                        if changeInDataset:
                            birth = line[122:126].strip()
                        else:
                            birth = line[126:130].strip()
                            
#                         if len(birth_str) > 0:
#                             birth = int(birth_str)
                        
                        if changeInDataset:
                            activity = line[128:].strip()
                        else:
                            activity = line[132:].strip()

                        if 'i' not in activity and len(rating_str) > 0 and rating > ratingLimit: # appending every entry to a list for sorting later
                            people.append((id, name, nat, rating_str, gender, games_played, birth, activity))
                            if rating > 2400 and id not in idList:
                                idList.append(id)                               
                            if id in idList:
                                nameRatingMap[(name, rating_str)] = id
                            if rating > 2400:
                                ratingsCounter[rating] += 1
                            ratingsCounterAll[rating] += 1
                    
                    people.sort(key=lambda tup: int(tup[0])) # sort all people by their FIDE identification 
                    for entry in people:
                        writeFile.write(entry[0] + "," + entry[1] + "," + entry[2] + "," + entry[3] + "," + entry[4] + "," + entry[5] + "," + entry[6] + "," + entry[7] + '\n')

            ratingsCounterList.append(ratingsCounter)
            ratingsCounterListAll.append(ratingsCounterAll)
#ratings in the constructed txt files are for the end of the month and should be used for building the next month's graph
dataPreprocessing()

In [43]:
# plotting graphs like a madman

def plotRatingDistributions(all = False):
    fig = go.Figure()
    i = 0
    ratingsToIterateOver = []
    if (all):
        ratingsToIterateOver = ratingsCounterListAll
    else:
        ratingsToIterateOver = ratingsCounterList
        
    for rc in ratingsToIterateOver:
        rc = sorted(rc.items(), key=itemgetter(0))
        fig.add_trace(go.Scattergl(x = [xc for xc, _ in rc], y = [yc for _, yc in rc] ,
                    mode='lines+markers',
                    name=monthDescriptiveLabel(i)))
        i = i + 1
    fig.show() 

In [44]:
plotRatingDistributions(True)

In [45]:
plotRatingDistributions(False)

In [5]:
rjecnikrjecnika = {}
for fideID in idList:
    rjecnikrjecnika[fideID] = {}
#fetching matches of a particular player commited for the specified month (many of these mathes were, possibly, played several months before, howbeit, were only commited now)

#id = FIDE id, date = string in yyyy-mm-01 format, returns adjacency list in form ("surname, name", "FIDE rating")
def fetchMatches(id, date):
    targetLink = "https://ratings.fide.com/individual_calculations.phtml?idnumber="+str(id)+"&rating_period="+date
    #page = requests.get("https://ratings.fide.com/individual_calculations.phtml?idnumber=2020009&rating_period=2015-11-01")
    page = requests.get(targetLink)
    soup = BeautifulSoup(page.content, 'html.parser')
    adjacencyDict = {}

    #fetch tournaments table "by rope and stick"
    tournament_table_tbody = soup.find('tr',  {"bgcolor": "#CC9966"})
    if tournament_table_tbody is None:
        return None
    tournament_table_tbody = tournament_table_tbody.parent
    
    rows = tournament_table_tbody.findAll("tr")
    tournament_rating = 0
    tournament_date = ''
    for tr in rows:
        if tr.has_attr('bgcolor'):
            if tr['bgcolor']=="#CC9966":               
                #for each tournament get month when it started
                tournament_date = tr.findAll('td')[3].contents[0].strip()
                tournament_date = tournament_date[:-2]+"01"
            if tr['bgcolor']=="#e6e6e6":
                tournament_rating = tr.findAll("td")[1].contents[0].strip()
                
        
        tdList = tr.findAll("td", {"class": "list4"})
        for td in tdList:
            if td.img:
                img_src = td.img.get('src')
                #print(img_src)
                if img_src == "/imga/clr_bl.gif":
                    color = "b"
                else: 
                    color = "w"
                td.img.decompose()

                cnt = 1
                for child in td.children:
                #print("Child ", cnt, ": ", child)
                    if cnt == 1:
                        name = child.strip()
                    if cnt == 4:
                        if child.font: #if * exists in the rating
                            rescaling = 1
                        else:
                            rescaling = 0
                        rating = child.contents[0].strip()
                    if cnt == 6:
                        result = child.contents[0].strip()
                    if cnt == 8:
                        transfer = child.contents[0].strip()
                    if cnt == 11:
                        if tournament_date in adjacencyDict.keys():
                            adjacencyDict[tournament_date].append((name, rating, tournament_rating, rescaling, color, result, transfer))
                        else:
                            adjacencyDict[tournament_date] = [(name, rating, tournament_rating, rescaling, color, result, transfer)]
                        
                        if rjecnikrjecnika[id].get(tournament_date) is None:
                            rjecnikrjecnika[id][tournament_date] = [(name, rating, tournament_rating, rescaling, color, result, transfer)]
                        else: 
                            rjecnikrjecnika[id][tournament_date].append((name, rating, tournament_rating, rescaling, color, result, transfer))
                    cnt = cnt + 1
    
    return adjacencyDict
    
# print(fetchMatches("2020009", "2015-11-01"))
# print(fetchMatches("1401815", "2018-01-01"))


In [6]:
def getMonthlyGraph(date):
    if date in monthlyGraphsHashMap.keys():
        return monthlyGraphsHashMap[date]
    else:
        G = nx.MultiGraph()
        for fideID in idList:
            G.add_node(str(fideID))
            
        G.add_node(str(dummyOver400Label))
#         G.add_node(str(dummyNotGMLabel))

        monthlyGraphsHashMap[date] = G
        return G
    
def saveMonthlyGraphs():
    for k, v in monthlyGraphsHashMap.items():
        nx.write_gml(v, os.path.join("MultiGraphsForMonths", k + "-graph.gml"))


def existsOppositeEdge(G, fideID, opponentID, eloGain, result, color):
    edges = G.get_edge_data(opponentID, fideID)
#     print(edges)
    if edges is not None:
        for _, attr in edges.items():
    #         print('prvi', attr['edgeSrc'] == opponentID)
    #         print('drugi', float(attr['ratingGainWhite']) == - float(eloGain))
    #         print('treci', float(attr['resultWhite']) == 1.0 - float(result))
    #         print('cetvrti', ((attr['whiteLabel'] == str(opponentID) and color == 'b') or (attr['whiteLabel'] == str(fideID) and color == 'w')))
            if attr['edgeSrc'] == opponentID and float(attr['ratingGainWhite']) == - float(eloGain) and float(attr['resultWhite']) == 1.0 - float(result) and \
            ((attr['whiteLabel'] == str(opponentID) and color == 'b') or (attr['whiteLabel'] == str(fideID) and color == 'w')):
                return True
        
    return False  
    
    
def buildMonthlyGraph(date):
    for fideID in tqdm(idList):
        gamesDict = fetchMatches(fideID, date)
        if gamesDict is not None:
            for month in gamesDict:
                G = getMonthlyGraph(month)
                for game in gamesDict[month]:
                    opponentName = game[0]
                    opponentRating = game[1]
                    opponentID = nameRatingMap.get((opponentName, opponentRating))
                    rating = game[2]
                    diffOver400 = game[3]
                    color = game[4]
                    result = game[5]
                    eloGain = game[6]

                    if opponentID is None and diffOver400 == 1:
                        opponentID = dummyOver400Label

                    if color == 'w':
                        if not existsOppositeEdge(G, str(fideID), str(opponentID), eloGain, result, color):
                            G.add_edge(str(fideID), str(opponentID), ratingGainWhite = eloGain, whiteLabel = str(fideID), blackLabel = str(opponentID), resultWhite = result, edgeSrc = str(fideID))
                    else:
                        if not existsOppositeEdge(G, str(fideID), str(opponentID), str(-float(eloGain)), str(1.0 - float(result)), color):
                            G.add_edge(str(fideID), str(opponentID), ratingGainWhite = str(-float(eloGain)), whiteLabel = str(opponentID), blackLabel = str(fideID), resultWhite = str(1.0 - float(result)), edgeSrc = str(fideID))
                
                
def buildAllMonthlyGraphs():
    for year in dataYears[::-1]:
        for month in dataMonths2Dig:
            buildMonthlyGraph(year + '-' + month + '-01')
            saveMonthlyGraphs()
            with open(os.path.join('MultiGraphsForMonths', 'crawler.pickle'), 'wb') as file:
                pickle.dump(rjecnikrjecnika, file)
            print('Done with', year, month)
            
def buildAllMonthlyGraphsTest():
    for year in dataYears[:1]:
        for month in dataMonths2Dig[:1]:
            buildMonthlyGraph("20" + year + '-' + month + '-01')
            
            folderName = "MultiGraphsForMonths"
            if not os.path.exists(folderName):
                os.makedirs(folderName)
            saveMonthlyGraphs()
            with open(os.path.join('MultiGraphsForMonths', 'crawler.pickle'), 'wb') as file:
                pickle.dump(rjecnikrjecnika, file)
            print('Done with', year, month)
            
#buildAllMonthlyGraphsTest()

In [55]:
G = nx.MultiGraph()
G.add_edge('0', '1', ratingGainWhite = '0.2', whiteLabel = '0', blackLabel = '1', resultWhite = '1.0', edgeSrc = '0')
G.add_edge('0', '1', ratingGainWhite = '0.0', whiteLabel = '1', blackLabel = '0', resultWhite = '0.5', edgeSrc = '0')

1

In [57]:
existsOppositeEdge(G, '0', '1', '0.0', '0.5', 'w')

{0: {'ratingGainWhite': '0.2', 'whiteLabel': '0', 'blackLabel': '1', 'resultWhite': '1.0', 'edgeSrc': '0'}, 1: {'ratingGainWhite': '0.0', 'whiteLabel': '1', 'blackLabel': '0', 'resultWhite': '0.5', 'edgeSrc': '0'}}
prvi False
drugi False
treci False
cetvrti True
prvi False
drugi True
treci True
cetvrti False


False